In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
plt.rc("font", family = 'Malgun Gothic')
plt.rc("axes", unicode_minus=False)

## 크롤링한 파일 로드

In [4]:
df = pd.read_csv("inflearn-event.csv")
df.shape

(100, 1)

In [5]:
df.head()

,text
0,2020년 목표: 프알못에서 프로젝트 마스터 되기 \n관심 있는 강의 : 스프링 ...
1,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...
2,"2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는..."
3,2020년 목표 : 리액트 공부하기
4,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급


In [6]:
df.tail()

,text
95,"사이버수사대(디지털포렌식)를 지원하고 싶은 현직 경찰관입니다. 현재는 아는 것도, ..."
96,"퇴사한지 2년 (헉 벌써…) 되가는 백수 개발자입니다. 현재 리액트, 노드, 파이어..."
97,"취업 성공~~!!!\n#관심강의: 스프링, 자바, Swift"
98,"올해는 파이썬, 데이터 분석, 머신러닝을 배워\n업무에 조금이라도 활용하고 싶습니다..."
99,오늘코드 사랑해요 ~ \n데이터 분석 기초를 둔둔하게~ \n#관심강의 : Oracl...


## 데이터 전처리 

### 중복 값 제거 

In [7]:
print(df.shape)
df = df.drop_duplicates(["text"], keep="last")
print(df.shape)

(100, 1)
(97, 1)


### 원본 따로 보존

In [8]:
df["original_text"] = df["text"]
df.head()

,text,original_text
0,2020년 목표: 프알못에서 프로젝트 마스터 되기 \n관심 있는 강의 : 스프링 ...,2020년 목표: 프알못에서 프로젝트 마스터 되기 \n관심 있는 강의 : 스프링 ...
1,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...,2020년 목표: 스프링 열심히 공부하서 직장에서 사랑받고 싶어요!!\n관심 있는 ...
2,"2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는...","2020년 목표: C++ 열심히 공부해서, 학교에서 꼭 A 맞기..!! \n관심있는..."
4,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급
5,2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!,2020년 목표 : 돌머리 비전공자가 멋진 전공자 되기!


### 소문자 변환
파이썬은 대소문자를 구분하기 때문에 데이터 필터링을 위해 대문자를 모두 소문자로 변경

In [9]:
df["text"] = df["text"].str.lower()

In [10]:
df["text"] = df["text"].str.replace("python","파이썬").str.replace("pandas", "판다스").str.replace("javascript","자바스크립트").str.replace("java","자바").str.replace("react","리액트")

### 관심 강의 분리

In [11]:
df['course'] = df['text'].apply(lambda x: x.split("관심강의")[-1])
df['course'] = df['course'].apply(lambda x: x.split("관심 강의")[-1])
df['course'] = df['course'].apply(lambda x: x.split("관심 강좌")[-1])
df['course'] = df['course'].str.replace(":","")
df["course"].tail()

95           디지털포렌식 자격증 2급 대비강좌
96        파이썬 데이터시각화 분석 실전 프로젝트
97               스프링, 자바, swift
98            파이썬, 데이터 분석, 머신러닝
99      oracle sql, 데이터 분석, 파이썬
Name: course, dtype: object

In [12]:
df[['text','course']].tail()

,text,course
95,"사이버수사대(디지털포렌식)를 지원하고 싶은 현직 경찰관입니다. 현재는 아는 것도, ...",디지털포렌식 자격증 2급 대비강좌
96,"퇴사한지 2년 (헉 벌써…) 되가는 백수 개발자입니다. 현재 리액트, 노드, 파이어...",파이썬 데이터시각화 분석 실전 프로젝트
97,"취업 성공~~!!!\n#관심강의: 스프링, 자바, swift","스프링, 자바, swift"
98,"올해는 파이썬, 데이터 분석, 머신러닝을 배워\n업무에 조금이라도 활용하고 싶습니다...","파이썬, 데이터 분석, 머신러닝"
99,오늘코드 사랑해요 ~ \n데이터 분석 기초를 둔둔하게~ \n#관심강의 : oracl...,"oracle sql, 데이터 분석, 파이썬"


### 띄어쓰기를 제거한 텍스트에서 키워트 추출

In [15]:
search_keyword = ['머신러닝', '딥러닝', '파이썬', '판다스', '공공데이터', 'django', '크롤링', '시각화', '데이터 분석',
                   '웹개발', '엑셀', 'c', '자바', '자바스크립트', 'node', 'vue', '리액트']

for keyword in search_keyword:
    df[keyword] = df['course'].str.contains(keyword)

In [17]:
df.tail()

,text,original_text,course,머신러닝,딥러닝,파이썬,판다스,공공데이터,django,크롤링,시각화,데이터 분석,웹개발,엑셀,c,자바,자바스크립트,node,vue,리액트
95,"사이버수사대(디지털포렌식)를 지원하고 싶은 현직 경찰관입니다. 현재는 아는 것도, ...","사이버수사대(디지털포렌식)를 지원하고 싶은 현직 경찰관입니다. 현재는 아는 것도, ...",디지털포렌식 자격증 2급 대비강좌,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
96,"퇴사한지 2년 (헉 벌써…) 되가는 백수 개발자입니다. 현재 리액트, 노드, 파이어...","퇴사한지 2년 (헉 벌써…) 되가는 백수 개발자입니다. 현재 리액트, 노드, 파이어...",파이썬 데이터시각화 분석 실전 프로젝트,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False
97,"취업 성공~~!!!\n#관심강의: 스프링, 자바, swift","취업 성공~~!!!\n#관심강의: 스프링, 자바, Swift","스프링, 자바, swift",False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
98,"올해는 파이썬, 데이터 분석, 머신러닝을 배워\n업무에 조금이라도 활용하고 싶습니다...","올해는 파이썬, 데이터 분석, 머신러닝을 배워\n업무에 조금이라도 활용하고 싶습니다...","파이썬, 데이터 분석, 머신러닝",True,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False
99,오늘코드 사랑해요 ~ \n데이터 분석 기초를 둔둔하게~ \n#관심강의 : oracl...,오늘코드 사랑해요 ~ \n데이터 분석 기초를 둔둔하게~ \n#관심강의 : Oracl...,"oracle sql, 데이터 분석, 파이썬",False,False,True,False,False,False,False,False,True,False,False,True,False,False,False,False,False


In [19]:
df_python = df[df['text'].str.contains("파이썬|공공데이터|판다스")].copy()
df_python.head()

,text,original_text,course,머신러닝,딥러닝,파이썬,판다스,공공데이터,django,크롤링,시각화,데이터 분석,웹개발,엑셀,c,자바,자바스크립트,node,vue,리액트
4,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! : 우리를 위한 프로그래밍 : 파이썬 중급,40대 프로그래밍 시작! 우리를 위한 프로그래밍 파이썬 중급,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
19,파이썬 데이터시각화 분석 실전 프로젝트 수강하고 싶어요,파이썬 데이터시각화 분석 실전 프로젝트 수강하고 싶어요,파이썬 데이터시각화 분석 실전 프로젝트 수강하고 싶어요,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False
22,머신러닝 제대로 배워보고 싶습니다.\n#관심강의 : 파이썬 머신러닝 완벽가이드,머신러닝 제대로 배워보고 싶습니다.\n#관심강의 : 파이썬 머신러닝 완벽가이드,파이썬 머신러닝 완벽가이드,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
26,"#관심강의: 프로그래밍 시작하기 : 파이썬 입문, 공공데이터로 파이썬 데이터 분석 ...","#관심강의: 프로그래밍 시작하기 : 파이썬 입문, 공공데이터로 파이썬 데이터 분석 ...","프로그래밍 시작하기 파이썬 입문, 공공데이터로 파이썬 데이터 분석 시작하기\n파...",False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False
30,자바기반 웹 개발자입니다. 데이터 분석에 많이 쓰이는 파이썬이 궁금합니다.\n#관심...,자바기반 웹 개발자입니다. 데이터 분석에 많이 쓰이는 Python이 궁금합니다.\n...,"프로그래밍 시작하기 파이썬 입문, 공공데이터로 파이썬 데이터 분석 시작하기",False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False


In [23]:
df[search_keyword].sum().sort_values(ascending=False)

c         18
리액트       15
파이썬       15
vue       11
자바         9
데이터 분석     6
node       5
머신러닝       5
자바스크립트     4
공공데이터      4
시각화        3
딥러닝        2
크롤링        1
엑셀         0
웹개발        0
django     0
판다스        0
dtype: int64

In [24]:
text = df.loc[(df['공공데이터'] == True), "text"]
for t in text:
    print("-"*20)
    print(t)

--------------------
#관심강의: 프로그래밍 시작하기 : 파이썬 입문, 공공데이터로 파이썬 데이터 분석 시작하기
파이썬의 고수가 되고싶어요
--------------------
자바기반 웹 개발자입니다. 데이터 분석에 많이 쓰이는 파이썬이 궁금합니다.
#관심강의: 프로그래밍 시작하기 : 파이썬 입문, 공공데이터로 파이썬 데이터 분석 시작하기
--------------------
올해 안에 원하는 공부 다 끝내보려고요. 내년이면 수능이라..

#관심강의: 공공데이터로 파이썬 데이터 분석 시작하기
--------------------
it개발자입니다. 개발을 오래 안하고 관리만 했더니 다시 개발이 너무 하고싶습니다. 올해는 인프런을 통해서 개발쪽 스킬셋을 업그레이드 하고 싶습니다!
#관심강의: 공공데이터로 파이썬 데이터 분석 시작하기


In [26]:
df.loc[df["공공데이터"] == True, "text"]

26    #관심강의: 프로그래밍 시작하기 : 파이썬 입문, 공공데이터로 파이썬 데이터 분석 ...
30    자바기반 웹 개발자입니다. 데이터 분석에 많이 쓰이는 파이썬이 궁금합니다.\n#관심...
53    올해 안에 원하는 공부 다 끝내보려고요. 내년이면 수능이라..\n\n#관심강의: 공...
93    it개발자입니다. 개발을 오래 안하고 관리만 했더니 다시 개발이 너무 하고싶습니다....
Name: text, dtype: object

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word',
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             min_df = 2,
                             ngram_range = (3,6),
                             max_features = 1000)

vectorizer

CountVectorizer(max_features=1000, min_df=2, ngram_range=(3, 6))

In [30]:
feature_vector = vectorizer.fit_transform(df['course'])
feature_vector.shape

(97, 113)

In [31]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

113


['2020년 목표 리액트',
 '2020년 목표 리액트 공부하기',
 'age of vue',
 'age of vue js',
 'api 개발과 성능',
 'api 개발과 성능 최적화',
 'deep reinforcement learning',
 'deep reinforcement learning 파이썬',
 'deep reinforcement learning 파이썬 머신러닝',
 'jpa 활용2 api']

In [32]:
pd.DataFrame(feature_vector[:10].toarray(), columns=vocab).head()

,2020년 목표 리액트,2020년 목표 리액트 공부하기,age of vue,age of vue js,api 개발과 성능,api 개발과 성능 최적화,deep reinforcement learning,deep reinforcement learning 파이썬,deep reinforcement learning 파이썬 머신러닝,jpa 활용2 api,...,파이썬 입문 공공데이터로 파이썬 데이터,파이썬 입문 공공데이터로 파이썬 데이터 분석,프로그래밍 시작하기 파이썬,프로그래밍 시작하기 파이썬 입문,프로그래밍 시작하기 파이썬 입문 공공데이터로,프로그래밍 시작하기 파이썬 입문 공공데이터로 파이썬,홍정모의 따라하며 배우는,활용2 api 개발과,활용2 api 개발과 성능,활용2 api 개발과 성능 최적화
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
